In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import os
import urllib
from retry import retry
import time
import pdb
from tqdm.notebook import tqdm 
import sqlite3
from ratelimiter import RateLimiter
import re
from ao3 import AO3
import sqlalchemy

In [89]:
!del kudos.db

C:\users\mcman\PycharmProjects\ao3_recs\notebooks\kudos.db


The process cannot access the file because it is being used by another process.


In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, JSON, ForeignKey, Table
from sqlalchemy.orm import relationship, Session, configure_mappers

In [3]:
engine = sqlalchemy.create_engine('sqlite:///kudos.db')
# engine = sqlalchemy.create_engine('sqlite:///:memory:')
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    __table_args__ = {'extend_existing': True} 

    id = Column(String, primary_key=True)
    works = relationship('Work', backref='author')

    def __repr__(self):
        return f"User(id='{self.id}')"

    
class Work(Base):
    __tablename__ = 'works'
    __table_args__ = {'extend_existing': True} 

    id = Column(Integer, primary_key=True)
    title = Column(String)
    author_id = Column(String, ForeignKey('users.id'))
#     json=Column(JSON)  TODO make this work
    
    def __repr__(self):
        return f"Work(id={repr(self.id)}, title={repr(self.title)})"

    kudos_left_by = relationship("User",
                    secondary=lambda: kudos_table,
                    backref='kudos')



kudos_table = Table('kudos', Base.metadata,
    Column('user', String, ForeignKey('users.id')),
    Column('work', Integer, ForeignKey('works.id')),
    extend_existing=True
)
Base.metadata.create_all(engine)

In [86]:
ratelimiter = RateLimiter(max_calls=10, period=30)

In [5]:
base_url = 'https://archiveofourown.org/'
base_fandom = 'https://archiveofourown.org/tags/{fandom}/works'
shera = 'She-Ra%20and%20the%20Princesses%20of%20Power%20(2018)'
base_qs = {'utf8': ['✓'],
 'commit': ['Sort and Filter'],
 'work_search[sort_column]': ['created_at'],
 'work_search[query]': ['sort:>posted'],
 'page': ['1']}
pages = 537

In [14]:
resp=requests.get(base_fandom.format(fandom=shera), params=base_qs)

soup = BeautifulSoup(resp.content)

works_on_page = [x['href'].split('/')[2] for x in soup.find_all('a', {'href':re.compile('^/works/\d+$')})]

ao3 = AO3()

work = ao3.work(works_on_page[0])

In [15]:
works_on_page

['20440244',
 '20440316',
 '20440652',
 '20441951',
 '20442869',
 '20442929',
 '20443265',
 '20443880',
 '20444141',
 '20444627',
 '20445464',
 '20450285',
 '20450966',
 '20453540',
 '20453750',
 '20454104',
 '20454308',
 '20454383',
 '20458088',
 '20458118']

In [6]:
ao3=AO3()

In [7]:
session = Session(bind=engine)

In [73]:
def get_or_create(session, model, **kwargs):
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return instance
    else:
        instance = model(**kwargs)
        session.add(instance)
        session.commit()
        return instance
    
def process_work(work_id):
    with ratelimiter:
        work = ao3.work(work_id)
    try:
        author = get_or_create(session, User, id=work.author)
        work_db = get_or_create(session, Work, id=int(work.id), 
                        title=work.title, 
                        author=author)
    except: # todo: fix author for works by anonymous (16623428)
        work_db = get_or_create(session, Work, id=int(work.id), 
                        title=work.title)
    
    kudos = list(work.kudos_left_by)
    existing_users_kudos = [x for x in session.query(User) if x.id in kudos]
    new_kudos = [User(id=x) for x in kudos if x not in (u.id for u in existing_users_kudos)]
    work_db.kudos_left_by = existing_users_kudos+new_kudos
    session.add(work_db)
    session.commit()

In [90]:
loaded = [str(x.id) for x in session.query(Work).all()]
with open('errors.txt', 'w') as ferr:
    for page in tqdm(range(1, pages+1)):
        base_qs['page'][0] = str(page)
        with ratelimiter:
            resp=requests.get(base_fandom.format(fandom=shera), params=base_qs)
        soup = BeautifulSoup(resp.content)
        works_on_page = [x['href'].split('/')[2] for x in soup.find_all('a', {'href':re.compile('^/works/\d+$')})]
#         print(len(works_on_page))
        for work in works_on_page:
            if work in loaded:
                continue
            try:
                process_work(work)
            except Exception as e:
                print(e, f"Work id: {work}")
                ferr.write(work+'\n')
                session.rollback()

'NoneType' object has no attribute 'contents' Work id: 16746358
'NoneType' object has no attribute 'contents' Work id: 16746598
'NoneType' object has no attribute 'contents' Work id: 17810264
'NoneType' object has no attribute 'contents' Work id: 17930453
'NoneType' object has no attribute 'contents' Work id: 17930486
'NoneType' object has no attribute 'contents' Work id: 17932343
'NoneType' object has no attribute 'contents' Work id: 18942466
'NoneType' object has no attribute 'contents' Work id: 18942490
'NoneType' object has no attribute 'contents' Work id: 18949180
'NoneType' object has no attribute 'contents' Work id: 18954202
'NoneType' object has no attribute 'contents' Work id: 18954670
'NoneType' object has no attribute 'contents' Work id: 18957028
'NoneType' object has no attribute 'contents' Work id: 18958240
'NoneType' object has no attribute 'contents' Work id: 18962710
'NoneType' object has no attribute 'contents' Work id: 18964225
'NoneType' object has no attribute 'cont

'NoneType' object has no attribute 'contents' Work id: 25457860
'NoneType' object has no attribute 'contents' Work id: 25457887
Work could not be loaded.
('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None)) Work id: 25458037
'NoneType' object has no attribute 'contents' Work id: 25458067
'NoneType' object has no attribute 'contents' Work id: 25458172
'NoneType' object has no attribute 'contents' Work id: 25458220
'NoneType' object has no attribute 'contents' Work id: 25458292
'NoneType' object has no attribute 'contents' Work id: 25458493
'NoneType' object has no attribute 'contents' Work id: 25458754
'NoneType' object has no attribute 'contents' Work id: 25459480
'NoneType' object has no attribute 'contents' Work id: 25459594
'NoneType' object has no attribute 'contents' Work id: 25461211
'N

In [91]:
session.query(Work).count()

1946

In [92]:
session.query(Work).all()[-1].kudos_left_by

[User(id='Somariel'),
 User(id='Catxdora'),
 User(id='Venn364'),
 User(id='Aries_Tenoh'),
 User(id='KnightsOfRayx'),
 User(id='ShadowAce29'),
 User(id='Ai220'),
 User(id='marivso'),
 User(id='Little_Chickadee'),
 User(id='Simpson17866'),
 User(id='Zaya_Ayame'),
 User(id='SeeASweetSmile'),
 User(id='little_red_panda'),
 User(id='Inopportunist'),
 User(id='Chezitcat'),
 User(id='Kenneday247'),
 User(id='Bluemidnight626'),
 User(id='Phoenix2210'),
 User(id='Southernprincess1776'),
 User(id='Nolarizendtari'),
 User(id='sonishko'),
 User(id='E2Shock'),
 User(id='WhentheWorldStoppedSpinning'),
 User(id='for_the_honor_of_grayskull_I_am_katniss'),
 User(id='Stephanie_the_Bi')]